In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

from preprocessing import preprocess
from walkforward import WFCVGenerator
from config.config_types import AppConfig

from utils.paths import CONFIG_DIR, SP500COPY_PATH, SP500_PATH, DATA_DIR

In [2]:
cfg = AppConfig.from_dict(f"{CONFIG_DIR}/vol_cnn.yaml")


In [3]:
wf = WFCVGenerator(cfg.walkforward)



16:46:55 - WFCVGenerator - DEBUG - WFConfig(train=3771d, val=1257d, test=1257d, lags=100, step=1257, max_folds=None) (walkforward.py:30)
16:46:55 - WFCVGenerator - DEBUG - self.scale_type: logstandard (walkforward.py:42)
16:46:55 - WFCVGenerator - DEBUG - self.scale: True (walkforward.py:43)
16:46:55 - WFCVGenerator - DEBUG - self.config.annualize: True (walkforward.py:58)
16:46:55 - Preprocessing - DEBUG - in preprocess (preprocessing.py:198)
16:46:55 - Preprocessing - DEBUG - In import data (preprocessing.py:13)
16:46:56 - Preprocessing - DEBUG - In handle nans (preprocessing.py:88)
16:46:56 - Preprocessing - DEBUG - In adjust for splits (preprocessing.py:51)
16:46:57 - Preprocessing - DEBUG - percentage of nan returns 0.0034% (preprocessing.py:100)
16:46:57 - Preprocessing - DEBUG - compute var (preprocessing.py:167)
16:46:57 - Preprocessing - DEBUG - number of nans in var: 6 (preprocessing.py:188)
16:46:57 - Preprocessing - DEBUG - create time index (preprocessing.py:158)
16:46:58 

In [4]:
for fold, data in enumerate(wf.folds()):
    print(fold)
    continue

use = data


Xtr             = use[0]
ytr             = use[1]
Xv              = use[2]
yv              = use[3]
Xte             = use[4]
yte             = use[5]
Xtr_val         = use[6]
ytr_val         = use[7]
Xte_merged      = use[8]
yte_merged      = use[9]
id_tr           = use[10]
id_v            = use[11]
id_te           = use[12]
window_train    = use[13]
window_val      = use[14]
window_test     = use[15]
X_scaler        = use[16]
y_scaler        = use[17]
X_scaler_merged = use[18]
y_scaler_merged = use[19]
    

16:47:07 - WFCVGenerator - DEBUG - output_cols: ['y'] (walkforward.py:388)
16:47:07 - WFCVGenerator - DEBUG - self.folds_count: 1 (walkforward.py:389)
16:47:14 - WFCVGenerator - DEBUG - self.config.clip: 0.1 (walkforward.py:414)
16:47:19 - WFCVGenerator - DEBUG - Shapes of Xtr, Xv, Xte before clipping:  (1798661, 100), (571977, 100), (575172, 100) (walkforward.py:440)
16:47:20 - WFCVGenerator - DEBUG - Fold 0 after clipping (y): train=(1723450, 1), val=(566956, 1), test=(564795, 1) (walkforward.py:451)
16:47:20 - WFCVGenerator - DEBUG - Fold 0 after clipping (X): train=(1723450, 100), val=(566956, 100), test=(564795, 100) (walkforward.py:454)
16:47:20 - WFCVGenerator - DEBUG - Scaling (walkforward.py:464)
16:47:30 - WFCVGenerator - DEBUG - Generating fold: 0 (walkforward.py:484)
16:47:30 - WFCVGenerator - DEBUG - Merged arrays shapes: Xtr_val=(2290406, 100), ytr_val=(2290406,) (walkforward.py:485)
16:47:30 - WFCVGenerator - DEBUG - Merged test shapes: Xte_merged=(564795, 100), yte_merg

0


In [5]:
X = np.concat([Xtr_val, Xte_merged])
Y = np.concat([ytr_val, yte_merged])

print(X.shape, Y.shape)

(2855201, 100) (2855201,)


In [7]:
import numpy as np

# 1. Add Bias Term (Intercept)
# Append a column of ones to X so the model can fit an intercept
X_b = np.c_[np.ones((X.shape[0], 1)), X]

# 2. Compute Beta (Weights)
# Using linalg.solve is numerically more stable than explicit inversion (inv)
# Equation: (X^T * X) * beta = X^T * Y
theta = np.linalg.solve(X_b.T @ X_b, X_b.T @ Y)

# 3. Predict
Y_pred = X_b @ theta

# 4. Calculate MSE
# MSE = (1/N) * sum((Y - Y_pred)^2)
mse = np.mean((Y - Y_pred) ** 2)


print(f"In-Sample MSE: {mse}")

In-Sample MSE: 0.4363541831265391


In [12]:
import numpy as np
import os

# Define your path (using the $DATASETS variable you set earlier is good practice)
save_path = os.path.join(DATA_DIR, 'processed_data_merged.npz')

# Save all 4 arrays into one file
np.savez(save_path, 
         X=X, 
         Y=Y)

---

In [55]:
df_preprocess = preprocess(portfolios=True)
df_preprocess

,date,var,permno,t
0,2000-01-31,0.000376,1,0
1,2000-01-31,0.000392,2,0
2,2000-01-31,0.000638,3,0
3,2000-01-31,0.000768,4,0
4,2000-01-31,0.000871,5,0
...,...,...,...,...
62695,2024-12-31,0.000099,6,6269
62696,2024-12-31,0.000114,7,6269
62697,2024-12-31,0.00012,8,6269
62698,2024-12-31,0.00018,9,6269
